## Question_12_Solution:

**Q-2.** A chemist had two chemical flasks labeled 0 and 1 which consist of two
different chemicals. He extracted 3 features from these chemicals in order to
distinguish between them, you provided the results derived by the chemicals and
your task is to create a model that will label chemical 0 or 1 given its three features
and built-in docker and use some library to display that in frontend.
Note : Use only pyspark

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

In [2]:
# Creating Pysaprk session:
spark = SparkSession.builder.appName("ChemicalClassifier").getOrCreate()

23/05/22 03:36:20 WARN Utils: Your hostname, ali-SVF15318SNW resolves to a loopback address: 127.0.1.1; using 192.168.208.116 instead (on interface wlp2s0)
23/05/22 03:36:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/22 03:36:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
# Load Indian Liver Patient Records:
data_df = spark.read.csv("useful/Q12.csv", header=True, inferSchema=True)
data_df.show(5)

+---+------+---------------+----------------+--------------------+------------------------+--------------------------+--------------+-------+--------------------------+-------+
|Age|Gender|Total_Bilirubin|Direct_Bilirubin|Alkaline_Phosphotase|Alamine_Aminotransferase|Aspartate_Aminotransferase|Total_Protiens|Albumin|Albumin_and_Globulin_Ratio|Dataset|
+---+------+---------------+----------------+--------------------+------------------------+--------------------------+--------------+-------+--------------------------+-------+
| 65|Female|            0.7|             0.1|                 187|                      16|                        18|           6.8|    3.3|                       0.9|      1|
| 62|  Male|           10.9|             5.5|                 699|                      64|                       100|           7.5|    3.2|                      0.74|      1|
| 62|  Male|            7.3|             4.1|                 490|                      60|                        

In [12]:
data_df.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Total_Bilirubin: double (nullable = true)
 |-- Direct_Bilirubin: double (nullable = true)
 |-- Alkaline_Phosphotase: integer (nullable = true)
 |-- Alamine_Aminotransferase: integer (nullable = true)
 |-- Aspartate_Aminotransferase: integer (nullable = true)
 |-- Total_Protiens: double (nullable = true)
 |-- Albumin: double (nullable = true)
 |-- Albumin_and_Globulin_Ratio: double (nullable = true)
 |-- Dataset: integer (nullable = true)



In [32]:
for col_name, col_type in data_df.dtypes:
    nullable = data_df.schema[col_name].nullable
    print("Column:", col_name)
    print("Nullable:", nullable)

Column: Age
Nullable: True
Column: Gender
Nullable: False
Column: Total_Bilirubin
Nullable: True
Column: Direct_Bilirubin
Nullable: True
Column: Alkaline_Phosphotase
Nullable: True
Column: Alamine_Aminotransferase
Nullable: True
Column: Aspartate_Aminotransferase
Nullable: True
Column: Total_Protiens
Nullable: True
Column: Albumin
Nullable: True
Column: Albumin_and_Globulin_Ratio
Nullable: True
Column: Dataset
Nullable: True
Column: Age_imputed
Nullable: True
Column: Total_Bilirubin_imputed
Nullable: True
Column: Direct_Bilirubin_imputed
Nullable: True
Column: Alkaline_Phosphotase_imputed
Nullable: True
Column: Alamine_Aminotransferase_imputed
Nullable: True
Column: Aspartate_Aminotransferase_imputed
Nullable: True
Column: Total_Protiens_imputed
Nullable: True
Column: Albumin_imputed
Nullable: True
Column: Albumin_and_Globulin_Ratio_imputed
Nullable: True


In [13]:
data_df.dtypes

[('Age', 'int'),
 ('Gender', 'string'),
 ('Total_Bilirubin', 'double'),
 ('Direct_Bilirubin', 'double'),
 ('Alkaline_Phosphotase', 'int'),
 ('Alamine_Aminotransferase', 'int'),
 ('Aspartate_Aminotransferase', 'int'),
 ('Total_Protiens', 'double'),
 ('Albumin', 'double'),
 ('Albumin_and_Globulin_Ratio', 'double'),
 ('Dataset', 'int')]

In [15]:
column_list = data_df.columns
column_list

['Age',
 'Gender',
 'Total_Bilirubin',
 'Direct_Bilirubin',
 'Alkaline_Phosphotase',
 'Alamine_Aminotransferase',
 'Aspartate_Aminotransferase',
 'Total_Protiens',
 'Albumin',
 'Albumin_and_Globulin_Ratio',
 'Dataset']

In [19]:
numerical_cols = []
categorical_cols = []

for col_name, col_type in data_df.dtypes:
    if col_type in ['byte', 'short', 'int', 'long', 'float', 'double']:
        numerical_cols.append(col_name)
    else:
        categorical_cols.append(col_name)

print("Numerical columns:", numerical_cols)
print("Categorical columns:", categorical_cols)


Numerical columns: ['Age', 'Total_Bilirubin', 'Direct_Bilirubin', 'Alkaline_Phosphotase', 'Alamine_Aminotransferase', 'Aspartate_Aminotransferase', 'Total_Protiens', 'Albumin', 'Albumin_and_Globulin_Ratio', 'Dataset']
Categorical columns: ['Gender']


In [53]:
from pyspark.sql.functions import when

data_df = data_df.withColumn("Gender", when(data_df["Gender"] == "Male", 0).otherwise(1))


In [21]:
from pyspark.ml.feature import Imputer

input_cols = ['Age',
 'Total_Bilirubin',
 'Direct_Bilirubin',
 'Alkaline_Phosphotase',
 'Alamine_Aminotransferase',
 'Aspartate_Aminotransferase',
 'Total_Protiens',
 'Albumin',
 'Albumin_and_Globulin_Ratio']

output_cols = ['Age_imputed',
 'Total_Bilirubin_imputed',
 'Direct_Bilirubin_imputed',
 'Alkaline_Phosphotase_imputed',
 'Alamine_Aminotransferase_imputed',
 'Aspartate_Aminotransferase_imputed',
 'Total_Protiens_imputed',
 'Albumin_imputed',
 'Albumin_and_Globulin_Ratio_imputed']

imputer = Imputer(strategy="mean", inputCols=input_cols, outputCols=output_cols)
data_df = imputer.fit(data_df).transform(data_df)


In [22]:
row_count = data_df.count()
print("Number of rows: ", row_count)

Number of rows:  583


In [23]:
data_df.select("Dataset").distinct().show()

+-------+
|Dataset|
+-------+
|      1|
|      2|
+-------+



In [27]:
data_df.select("Gender").distinct().show()

+------+
|Gender|
+------+
|     1|
|     0|
+------+



In [28]:
data_df.select('Age').distinct().show()

+---+
|Age|
+---+
| 31|
| 85|
| 65|
| 53|
| 78|
| 34|
| 28|
| 26|
| 27|
| 44|
| 12|
| 22|
| 47|
| 52|
| 13|
|  6|
| 16|
| 40|
| 20|
| 57|
+---+
only showing top 20 rows



In [29]:
numerical_cols = []
categorical_cols = []

for col_name, col_type in data_df.dtypes:
    if col_type in ['byte', 'short', 'int', 'long', 'float', 'double']:
        numerical_cols.append(col_name)
    else:
        categorical_cols.append(col_name)

print("Numerical columns:", numerical_cols)
print("Categorical columns:", categorical_cols)


Numerical columns: ['Age', 'Gender', 'Total_Bilirubin', 'Direct_Bilirubin', 'Alkaline_Phosphotase', 'Alamine_Aminotransferase', 'Aspartate_Aminotransferase', 'Total_Protiens', 'Albumin', 'Albumin_and_Globulin_Ratio', 'Dataset', 'Age_imputed', 'Total_Bilirubin_imputed', 'Direct_Bilirubin_imputed', 'Alkaline_Phosphotase_imputed', 'Alamine_Aminotransferase_imputed', 'Aspartate_Aminotransferase_imputed', 'Total_Protiens_imputed', 'Albumin_imputed', 'Albumin_and_Globulin_Ratio_imputed']
Categorical columns: []


In [31]:
data_df.dtypes

[('Age', 'int'),
 ('Gender', 'int'),
 ('Total_Bilirubin', 'double'),
 ('Direct_Bilirubin', 'double'),
 ('Alkaline_Phosphotase', 'int'),
 ('Alamine_Aminotransferase', 'int'),
 ('Aspartate_Aminotransferase', 'int'),
 ('Total_Protiens', 'double'),
 ('Albumin', 'double'),
 ('Albumin_and_Globulin_Ratio', 'double'),
 ('Dataset', 'int'),
 ('Age_imputed', 'int'),
 ('Total_Bilirubin_imputed', 'double'),
 ('Direct_Bilirubin_imputed', 'double'),
 ('Alkaline_Phosphotase_imputed', 'int'),
 ('Alamine_Aminotransferase_imputed', 'int'),
 ('Aspartate_Aminotransferase_imputed', 'int'),
 ('Total_Protiens_imputed', 'double'),
 ('Albumin_imputed', 'double'),
 ('Albumin_and_Globulin_Ratio_imputed', 'double')]

In [47]:
data_df = data_df.withColumn("Age", data_df["Age"].cast("double"))

In [54]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

# Create a StringIndexer to convert the "Gender" column into numeric values
stringIndexer = StringIndexer(inputCol="Gender", outputCol="GenderIndex")
data_df = stringIndexer.fit(data_df).transform(data_df)

# Create a OneHotEncoder to encode the "GenderIndex" column
oneHotEncoder = OneHotEncoder(inputCols=["GenderIndex"], outputCols=["GenderEncoded"])
data_df = oneHotEncoder.fit(data_df).transform(data_df)


IllegalArgumentException: requirement failed: The input column GenderIndex should have at least two distinct values.

In [48]:
# split fatures-target:
feature_columns = ['Age', 'Gender', 'Total_Bilirubin', 'Direct_Bilirubin', 'Alkaline_Phosphotase', 'Alamine_Aminotransferase', 'Aspartate_Aminotransferase', 'Total_Protiens', 'Albumin', 'Albumin_and_Globulin_Ratio', 'Age_imputed', 'Total_Bilirubin_imputed', 'Direct_Bilirubin_imputed', 'Alkaline_Phosphotase_imputed', 'Alamine_Aminotransferase_imputed', 'Aspartate_Aminotransferase_imputed', 'Total_Protiens_imputed', 'Albumin_imputed', 'Albumin_and_Globulin_Ratio_imputed']
target_column = 'Dataset'

In [55]:
from pyspark.ml.feature import VectorAssembler

# Assemble the features into a vector column
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data_with_features = assembler.transform(data_df).select("features", "Dataset")

# Split the data into training and testing sets
(training_data, testing_data) = data_with_features.randomSplit([0.7, 0.3], seed=123)


IllegalArgumentException: Output column Dataset already exists.

In [50]:
# Create an instance of the Random Forest classifier
rf = RandomForestClassifier(labelCol=target_column, featuresCol="features")

# Train the model using the training data
model = rf.fit(training_data)


23/05/22 04:10:17 ERROR Executor: Exception in task 0.0 in stage 43.0 (TID 35)
org.apache.spark.SparkException: Failed to execute user defined function (VectorAssembler$$Lambda$3742/0x000000084114b840: (struct<Age:double,Gender_double_VectorAssembler_da594bc9642c:double,Total_Bilirubin:double,Direct_Bilirubin:double,Alkaline_Phosphotase_double_VectorAssembler_da594bc9642c:double,Alamine_Aminotransferase_double_VectorAssembler_da594bc9642c:double,Aspartate_Aminotransferase_double_VectorAssembler_da594bc9642c:double,Total_Protiens:double,Albumin:double,Albumin_and_Globulin_Ratio:double,Age_imputed_double_VectorAssembler_da594bc9642c:double,Total_Bilirubin_imputed:double,Direct_Bilirubin_imputed:double,Alkaline_Phosphotase_imputed_double_VectorAssembler_da594bc9642c:double,Alamine_Aminotransferase_imputed_double_VectorAssembler_da594bc9642c:double,Aspartate_Aminotransferase_imputed_double_VectorAssembler_da594bc9642c:double,Total_Protiens_imputed:double,Albumin_imputed:double,Albumin_and_

Py4JJavaError: An error occurred while calling o657.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 43.0 failed 1 times, most recent failure: Lost task 0.0 in stage 43.0 (TID 35) (192.168.208.116 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (VectorAssembler$$Lambda$3742/0x000000084114b840: (struct<Age:double,Gender_double_VectorAssembler_da594bc9642c:double,Total_Bilirubin:double,Direct_Bilirubin:double,Alkaline_Phosphotase_double_VectorAssembler_da594bc9642c:double,Alamine_Aminotransferase_double_VectorAssembler_da594bc9642c:double,Aspartate_Aminotransferase_double_VectorAssembler_da594bc9642c:double,Total_Protiens:double,Albumin:double,Albumin_and_Globulin_Ratio:double,Age_imputed_double_VectorAssembler_da594bc9642c:double,Total_Bilirubin_imputed:double,Direct_Bilirubin_imputed:double,Alkaline_Phosphotase_imputed_double_VectorAssembler_da594bc9642c:double,Alamine_Aminotransferase_imputed_double_Ve

In [51]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

# Split features and target
feature_columns = ['Age', 'Gender', 'Total_Bilirubin', 'Direct_Bilirubin', 'Alkaline_Phosphotase', 'Alamine_Aminotransferase', 'Aspartate_Aminotransferase', 'Total_Protiens', 'Albumin', 'Albumin_and_Globulin_Ratio', 'Age_imputed', 'Total_Bilirubin_imputed', 'Direct_Bilirubin_imputed', 'Alkaline_Phosphotase_imputed', 'Alamine_Aminotransferase_imputed', 'Aspartate_Aminotransferase_imputed', 'Total_Protiens_imputed', 'Albumin_imputed', 'Albumin_and_Globulin_Ratio_imputed']
target_column = 'Dataset'

# Assemble the features into a vector column
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data_with_features = assembler.transform(data_df).select("features", "Dataset")

# Split the data into training and testing sets
(training_data, testing_data) = data_with_features.randomSplit([0.7, 0.3], seed=123)

# Create an instance of the Random Forest classifier
rf = RandomForestClassifier(labelCol=target_column, featuresCol="features")

# Train the model using the training data
model = rf.fit(training_data)


23/05/22 04:10:23 ERROR Executor: Exception in task 0.0 in stage 44.0 (TID 36)
org.apache.spark.SparkException: Failed to execute user defined function (VectorAssembler$$Lambda$3742/0x000000084114b840: (struct<Age:double,Gender_double_VectorAssembler_af86090d5711:double,Total_Bilirubin:double,Direct_Bilirubin:double,Alkaline_Phosphotase_double_VectorAssembler_af86090d5711:double,Alamine_Aminotransferase_double_VectorAssembler_af86090d5711:double,Aspartate_Aminotransferase_double_VectorAssembler_af86090d5711:double,Total_Protiens:double,Albumin:double,Albumin_and_Globulin_Ratio:double,Age_imputed_double_VectorAssembler_af86090d5711:double,Total_Bilirubin_imputed:double,Direct_Bilirubin_imputed:double,Alkaline_Phosphotase_imputed_double_VectorAssembler_af86090d5711:double,Alamine_Aminotransferase_imputed_double_VectorAssembler_af86090d5711:double,Aspartate_Aminotransferase_imputed_double_VectorAssembler_af86090d5711:double,Total_Protiens_imputed:double,Albumin_imputed:double,Albumin_and_

Py4JJavaError: An error occurred while calling o736.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 44.0 failed 1 times, most recent failure: Lost task 0.0 in stage 44.0 (TID 36) (192.168.208.116 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (VectorAssembler$$Lambda$3742/0x000000084114b840: (struct<Age:double,Gender_double_VectorAssembler_af86090d5711:double,Total_Bilirubin:double,Direct_Bilirubin:double,Alkaline_Phosphotase_double_VectorAssembler_af86090d5711:double,Alamine_Aminotransferase_double_VectorAssembler_af86090d5711:double,Aspartate_Aminotransferase_double_VectorAssembler_af86090d5711:double,Total_Protiens:double,Albumin:double,Albumin_and_Globulin_Ratio:double,Age_imputed_double_VectorAssembler_af86090d5711:double,Total_Bilirubin_imputed:double,Direct_Bilirubin_imputed:double,Alkaline_Phosphotase_imputed_double_VectorAssembler_af86090d5711:double,Alamine_Aminotransferase_imputed_double_Ve